In [15]:
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform

In [16]:
# 엑셀 파일을 데이터프레임으로 불러오기
excel_file_path = r"./geocode_welfare.xlsx"
df_wel = pd.read_excel(excel_file_path)
df_wel

,시군구명,시설명,시설주소,경도,위도,시설종류명(시설유형),정원(수용인원),현인원
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),127.097852,37.505381,(장애인) 장애인복지관,160.0,160.0
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,127.094073,37.506119,(장애인) 장애인복지관,0.0,0.0
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),126.919712,37.493234,(장애인) 장애인복지관,752.0,752.0
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,127.042358,37.509405,(장애인) 장애인복지관,NaN,345.0
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),126.909068,37.563706,(장애인) 장애인복지관,NaN,3743.0
...,...,...,...,...,...,...,...,...
672,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),126.984944,37.570812,(장애인) (기타)장애인복지시설,30.0,NaN
673,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",126.898513,37.521684,(장애인) (기타)장애인복지시설,NaN,NaN
674,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",127.039177,37.500322,(장애인) (기타)장애인복지시설,24.0,NaN
675,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",127.046317,37.673059,(장애인) (기타)장애인복지시설,30.0,NaN


In [17]:
null_rows = df_wel[df_wel['경도'].isnull()]
null_rows

,시군구명,시설명,시설주소,경도,위도,시설종류명(시설유형),정원(수용인원),현인원
271,마포구,마포장애인주간보호센터,서울특별시 마포구 성산로4길 35마포장애인종합복지관 (성산동),NaN,NaN,(장애인) 장애인주간보호시설,12.0,10.0
574,성동구,성동재활의원,"서울특별시 성동구 뚝섬로1길 43, 1층 (성수동1가, 성수문화복지회관)",NaN,NaN,(장애인) 장애인의료재활시설,30.0,30.0
616,양천구,양천희망돌봄센터,"서울특별시 양천구 가로공원로 101 (신월동, 신월누리복지센터)",NaN,NaN,(장애인) 장애인주간보호시설,10.0,10.0


In [18]:
# null_rows에 위도와 경도 값을 직접 추가
df_wel.loc[271, ['위도', '경도']] = 37.563786147668246, 126.90887964298143
df_wel.loc[574, ['위도', '경도']] = 37.54542035909292, 127.04695216433396
df_wel.loc[616, ['위도', '경도']] = 37.53592391553605, 126.8272378979522

In [19]:
df_wel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   시군구명         677 non-null    object 
 1   시설명          677 non-null    object 
 2   시설주소         677 non-null    object 
 3   경도           677 non-null    float64
 4   위도           677 non-null    float64
 5   시설종류명(시설유형)  677 non-null    object 
 6   정원(수용인원)     627 non-null    float64
 7   현인원          628 non-null    float64
dtypes: float64(4), object(4)
memory usage: 42.4+ KB


In [20]:
# '위도'와 '경도' 컬럼의 좌표계를 'EPSG:4326'에서 'EPSG:5186'로 변환
source_crs = 'EPSG:4326'
target_crs = 'EPSG:5186'

In [21]:
# 좌표 변환 함수
def convert_coordinates(row):
    lon, lat = row['경도'], row['위도']
    x, y = transform(Proj(init=source_crs), Proj(init=target_crs), lon, lat)
    return pd.Series({'X': x, 'Y': y})

# '위도'와 '경도' 컬럼을 'EPSG:4326'에서 'EPSG:5186'로 변환
df_wel[['X', 'Y']] = df_wel.apply(convert_coordinates, axis=1)

# '위도'와 '경도' 컬럼 삭제
df_wel = df_wel.drop(['위도', '경도'], axis=1)

C:\Users\yeeun\AppData\Roaming\Python\Python311\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\yeeun\AppData\Roaming\Python\Python311\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\yeeun\AppData\Local\Temp\ipykernel_15540\2413262106.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-

In [22]:
# 결과 확인
df_wel

,시군구명,시설명,시설주소,시설종류명(시설유형),정원(수용인원),현인원,X,Y
0,송파구,송파구방이복지관,서울특별시 송파구 백제고분로37길 4 (석촌동)(석촌동),(장애인) 장애인복지관,160.0,160.0,208651.969061,545105.809833
1,송파구,서울시각장애인복지관,서울특별시 송파구 삼학사로19길 27,(장애인) 장애인복지관,0.0,0.0,208317.742410,545187.410939
2,동작구,서울시립발달장애인복지관,서울특별시 동작구 여의대방로20길 33-0395(보라매공원내),(장애인) 장애인복지관,752.0,752.0,192899.812595,543756.235789
3,강남구,청음복지관,서울특별시 강남구 봉은사로50길 6,(장애인) 장애인복지관,NaN,345.0,203745.064042,545548.855807
4,마포구,마포장애인종합복지관,서울특별시 마포구 성산로4길 353층(마포장애인종합복지관)(성산동),(장애인) 장애인복지관,NaN,3743.0,191966.141651,551578.632818
...,...,...,...,...,...,...,...,...
672,종로구,종로발달장애인평생교육센터,서울특별시 종로구 종로17길 8 (종로2가),(장애인) (기타)장애인복지시설,30.0,NaN,198669.960093,552363.502415
673,영등포구,영등포구 발달장애인평생교육센터,"서울특별시 영등포구 문래북로 1055~7층, 어울림센터 (당산동1가)",(장애인) (기타)장애인복지시설,NaN,NaN,191028.534919,546915.621233
674,강남구,강남발달장애인평생교육센터,"서울특별시 강남구 논현로86길 21, 2층 (역삼동)",(장애인) (기타)장애인복지시설,24.0,NaN,203464.246509,544540.571221
675,도봉구,도봉발달장애인 평생교육센터,"서울특별시 도봉구 마들로 703, 4~6층 (도봉동)",(장애인) (기타)장애인복지시설,30.0,NaN,204086.095211,563712.753940


In [26]:
csv_file_path = r"./geocode_welfare_transtormed.csv"

df_wel.to_csv(csv_file_path, index=False)